In [ ]:
from mlflow.tracking import MlflowClient

#mlflow server --backend-store-uri sqlite:///mlflow.db
ML_FLOW_TRACKING_URL = "sqlite:///mlflow.db"

client = MlflowClient(tracking_uri=ML_FLOW_TRACKING_URL)

In [17]:
client.search_experiments()

[<Experiment: artifact_location='/workspaces/mlops-course/02-experiment-tracking/mlruns/1', creation_time=1768224496122, experiment_id='1', last_update_time=1768224496122, lifecycle_stage='active', name='nyc-taxi-experiment', tags={'mlflow.experimentKind': 'custom_model_development'}>]

In [18]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse > 6.6",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=10,
    order_by=['metrics.rmse ASC']

)

In [19]:
for run in runs:
    print(f'run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}')

run id: a0f62b7d1bb242b3930ac0e81aab2257, rmse: 6.7423
run id: b93ff0ad12d54a9e8d8e36a138f16727, rmse: 6.9198
run id: 9cc4e6c9c5614e37a78c50aff5e51603, rmse: 6.9275
run id: 184b27e50d034e95934d7d59f51248bb, rmse: 12.1432
run id: 4b372a9dcb234d5c9d6a66c4d22616bf, rmse: 839.7729


In [22]:
import mlflow

mlflow.set_tracking_uri(ML_FLOW_TRACKING_URL)

run_id = 'a0f62b7d1bb242b3930ac0e81aab2257'
model_uri = f'runs:/{run_id}/model'
mlflow.register_model(model_uri=model_uri, name='nyc-taxi-regressor')

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
2026/01/12 20:58:02 WARNING mlflow.tracking._model_registry.fluent: Run with id a0f62b7d1bb242b3930ac0e81aab2257 has no artifacts at artifact path 'model', registering model based on models:/m-65a81da224994a7a85494310bc96b500 instead
Created version '3' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1768247882656, current_stage='None', deployment_job_state=None, description=None, last_updated_timestamp=1768247882656, metrics=None, model_id=None, name='nyc-taxi-regressor', params=None, run_id='a0f62b7d1bb242b3930ac0e81aab2257', run_link=None, source='models:/m-65a81da224994a7a85494310bc96b500', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [51]:
model_name = 'nyc-taxi-regressor'
client.get_latest_versions(name=model_name)

C:\Users\barto\AppData\Local\Temp\ipykernel_13608\1965448924.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.get_latest_versions(name=model_name)


[<ModelVersion: aliases=[], creation_timestamp=1768247882656, current_stage='None', deployment_job_state=None, description=None, last_updated_timestamp=1768247882656, metrics=None, model_id=None, name='nyc-taxi-regressor', params=None, run_id='a0f62b7d1bb242b3930ac0e81aab2257', run_link=None, source='models:/m-65a81da224994a7a85494310bc96b500', status='READY', status_message=None, tags={}, user_id=None, version=3>]

In [52]:
import pandas as pd
from sklearn.metrics import root_mean_squared_error

def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, version, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{version}")
    y_pred = model.predict(X_test)
    return {"rmse": root_mean_squared_error(y_test, y_pred)}

In [27]:
df = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-03.parquet')

In [28]:
client.download_artifacts(run_id=run_id, path='preprocessor',dst_path='.')

'c:\\Users\\barto\\mlops-course\\mlops-course\\02-experiment-tracking\\preprocessor'

In [29]:
import pickle

with open('preprocessor/preprocessor.b','rb') as f_in:
    dv = pickle.load(f_in)

In [ ]:
X_test = preprocess(df,dv)

In [31]:
target = 'duration'
y_test = df[target].values


In [53]:
%time test_model(name=model_name, version=3, X_test=X_test, y_test=y_test)

CPU times: total: 1.73 s
Wall time: 3.85 s


{'rmse': 6.659623830022514}